In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib
import plotly.express as px
from datetime import datetime
from datastory import DataStory

sys.path.append('../lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(utils)
importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

utils.set_secrets_as_env()

# Krav

In [ ]:
con = pesys_utils.open_pen_connection()
df_krav = pandas_utils.pandas_from_sql('../sql/krav_med_kontrollpunkt.sql', con, tuning=1000, lowercase=True)
con.close()

In [ ]:
df_krav.head()

In [ ]:
df1 = df_krav.copy()
df1["ar-mnd"] = df1.apply(lambda row: str(int(row.yr)) + "-" + str(int(row.mnd)), axis=1)
df1["ar-mnd"] = df1["ar-mnd"].apply(lambda x: pesys_utils.add_zero_to_aar_mnd(x))
df1["tertial"] = df1.mnd.apply(lambda x: (x - 1) // 4 + 1)
df1["ar-tertial"] = df1.apply(lambda row: str(row.yr) + "-" + str(row.tertial), axis=1)
df1.head()

In [ ]:
df_krav_count = df1.groupby(["ar-tertial"], as_index=False).antall.sum()
df_krav_count.head()

In [ ]:
df1_tertial = df1.groupby(["ar-tertial", "tertial", "yr", "kontrollpunkt", "kontrollpunkt_forklaring"], as_index=False).antall.sum().sort_values("ar-tertial")
df1_tertial = df1_tertial.merge(df_krav_count, on="ar-tertial").sort_values("ar-tertial")
df1_tertial["andel"] = df1_tertial.antall_x / df1_tertial.antall_y
df1_tertial.head()

In [ ]:
figs = {}
for kp in df1_tertial.kontrollpunkt.unique():
    df_plot = df1_tertial[(df1_tertial.yr >= 2020) & (df1_tertial.kontrollpunkt == kp)]
    figs[kp] = px.line(df_plot, "tertial", "andel", color="yr", title=kp).update_layout(xaxis_type='category').update_yaxes(tickformat='.1%')

In [ ]:
for _, fig in figs.items():
    fig.show()

# Delauto (sjekk sql!!!)

In [ ]:
con = pesys_utils.open_pen_connection()
df_ora = pandas_utils.pandas_from_sql('../sql/kontrollpunkt_delauto.sql', con, tuning=1000, lowercase=True)
con.close()

In [ ]:
df = df_ora.copy()
df["ar-mnd"] = df.apply(lambda row: str(row.yr) + "-" + str(row.mnd), axis=1)
df["ar-mnd"] = df["ar-mnd"].apply(lambda x: pesys_utils.add_zero_to_aar_mnd(x))
df["tertial"] = df.mnd.apply(lambda x: (x - 1) // 4 + 1)
df["ar-tertial"] = df.apply(lambda row: str(row.yr) + "-" + str(row.tertial), axis=1)
df.head()

In [ ]:
df_kp = df.groupby(["ar-mnd", "kontrollpunkt", "kontrollpunkt_forklaring"], as_index=False).antall.sum().sort_values("ar-mnd")
df_kp.head()

In [ ]:
px.line(df_kp, "ar-mnd", "antall", color="kontrollpunkt", hover_data="kontrollpunkt_forklaring")

In [ ]:
df_tertial = df.groupby(["ar-tertial", "tertial", "yr", "kontrollpunkt", "kontrollpunkt_forklaring"], as_index=False).antall.sum().sort_values("ar-tertial")

figs = {}
for kp in df_tertial.kontrollpunkt.unique():
    df_plot = df_tertial[(df_tertial.yr >= 2020) & (df_tertial.kontrollpunkt == kp)]
    figs[kp] = px.line(df_plot, "tertial", "antall", color="yr", title=kp).update_layout(xaxis_type='category')

In [ ]:
for _, fig in figs.items():
    fig.show()

# Manuell

In [ ]:
con = pesys_utils.open_pen_connection()
df_man = pandas_utils.pandas_from_sql('../sql/kontrollpunkt_man.sql', con, tuning=1000, lowercase=True)
con.close()

In [ ]:
df_2 = df_man.copy()
df_2["ar-mnd"] = df_2.apply(lambda row: str(row.yr) + "-" + str(row.mnd), axis=1)
df_2["ar-mnd"] = df_2["ar-mnd"].apply(lambda x: pesys_utils.add_zero_to_aar_mnd(x))
df_2["tertial"] = df_2.mnd.apply(lambda x: (x - 1) // 4 + 1)
df_2["ar-tertial"] = df_2.apply(lambda row: str(row.yr) + "-" + str(row.tertial), axis=1)
df_2.head()

In [ ]:
df_kp_2 = df_2.groupby(["ar-mnd", "kontrollpunkt", "kontrollpunkt_forklaring"], as_index=False).antall.sum().sort_values("ar-mnd")
df_kp_2.head()

In [ ]:
px.line(df_kp_2, "ar-mnd", "antall", color="kontrollpunkt", hover_data="kontrollpunkt_forklaring")

In [ ]:
df_tertial = df_2.groupby(["ar-tertial", "tertial", "yr", "kontrollpunkt", "kontrollpunkt_forklaring"], as_index=False).antall.sum().sort_values("ar-tertial")

figs_2 = {}
for kp in df_tertial.kontrollpunkt.unique():
    df_plot = df_tertial[(df_tertial.yr >= 2020) & (df_tertial.kontrollpunkt == kp)]
    #df_plot["antall"] = df_plot.apply(lambda row: float(row.antall) * 1.14 if row.yr==2023 and row.tertial==2 else float(row.antall), axis=1)
    kp_forklaring = df_plot.kontrollpunkt_forklaring.iloc[0]
    figs_2[kp] = px.line(df_plot, "tertial", "antall", color="yr", title=kp_forklaring).update_layout(xaxis_type='category')

In [ ]:
for _, fig in figs_2.items():
    fig.show()